In [2]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder

In [3]:
train_goods = pd.concat([
    pd.read_csv('./训练集/traindata_goodsid/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00002', header=None, names=['goods_id', 'cat_id', 'brandsn'])
], axis=0)

train_user = pd.concat([
    pd.read_csv(x, header=None, names=['user_id', 'goods_id', 'is_clk', 'is_like', 'is_addcart', 'is_order', 'expose_start_time', 'dt'], nrows=500000)
    for x in glob.glob('./训练集/traindata_user/part*')
], axis=0)

In [4]:
testa_goods = pd.concat([
    pd.read_csv('./测试集a/predict_goods_id/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./测试集a/predict_goods_id/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
], axis=0)

testa_user = pd.read_excel('./测试集a/a榜需要预测的uid_5000.xlsx')

In [5]:
#对训练集中的用户ID和商品ID进行标签编码
user_encode = LabelEncoder()
user_encode.fit(list(train_user['user_id']) + list(train_user['user_id']))

goods_encode = LabelEncoder()
goods_encode.fit(list(train_user['goods_id']) + list(train_goods['goods_id']) + list(train_goods['goods_id']))

LabelEncoder()

In [6]:
#计算测试集用户数据中的用户ID和商品ID在训练集用户数据中出现的比例
np.mean(testa_user['user_id'].isin(train_user['user_id'])), np.mean(testa_goods['goods_id'].isin(train_goods['goods_id']))

(1.0, 0.9999751455447659)

In [7]:
#去重
train_user['user_id'].nunique(), train_user['goods_id'].nunique()

(51602, 1531577)

In [8]:
#计算训练集用户数据的描述统计信息，并将结果保留两位小数
train_user.describe().round(2)

,is_clk,is_like,is_addcart,is_order,dt
count,7791816.00,7791816.00,7791816.00,7791816.00,7791816.00
mean,1.06,0.02,0.06,0.01,20230222.85
std,0.28,0.13,0.27,0.09,26.21
min,1.00,0.00,0.00,0.00,20230203.00
25%,1.00,0.00,0.00,0.00,20230209.00
50%,1.00,0.00,0.00,0.00,20230216.00
75%,1.00,0.00,0.00,0.00,20230224.00
max,121.00,22.00,28.00,15.00,20230303.00


In [9]:
#将训练集中的用户数据和商品数据根据商品ID进行合并
train_data = pd.merge(train_user.iloc[:], train_goods.iloc[:], on='goods_id')

In [10]:
#获取合并后的数据集中商品类别和品牌的数量
train_data['cat_id'].nunique(), train_data['brandsn'].nunique()
print(train_data)

                                  user_id                          goods_id  \
0        8da2ec07d8bf9bfe1e849cb7e7f25e5c  f6e4f43d18157cbdcdc653c6e35f01fb   
1        62a800cd3e6884078ac86af31926ca6e  f6e4f43d18157cbdcdc653c6e35f01fb   
2        dfea38a0a8c81429316857cbcede23c6  f6e4f43d18157cbdcdc653c6e35f01fb   
3        dfea38a0a8c81429316857cbcede23c6  f6e4f43d18157cbdcdc653c6e35f01fb   
4        328395043a7cf38a36590a8df12b3b03  f6e4f43d18157cbdcdc653c6e35f01fb   
...                                   ...                               ...   
7791759  9b9c0493e6ee675c32b6c2add2b57cd9  49cb5e43cc1dc19cb5f5241cf06816a5   
7791760  2591ff4a1af75a2dc9e020cdafc5ab05  7fb4a90d3aa54ce13355c8ad9043e2e1   
7791761  18917649fd13d3a5fc37443c248d572e  a7e398eff0f997168bf341235ea4817d   
7791762  a4b0bc37d9cceed3bedc794e8d632d38  d6f2d674b397f89a55ecf34f127d0ad3   
7791763  333cb2e2e4c37293ca79f985a66d72f4  2acf9dbcd66278505f54da19d636825c   

         is_clk  is_like  is_addcart  is_order    e

In [11]:
#筛选出合并后的训练数据中满足以下条件的样本：未下单（is_order为0）且加购物车行为不为0（is_addcart不等于0）
train_agg_feat = train_data.loc[
    (train_data['is_order'] == 0) & (train_data['is_addcart'] != 0)
]
print(train_agg_feat)

                                  user_id                          goods_id  \
19       af640ab2869a652c05b0957d9a811704  81de1226af20e196944862b78d27b0b5   
76       11925e60aab947c3b00eee2190fa2ede  ff76680bd6e705fd37df578957faacba   
191      c626a0405a9d85abdf05ccfd3d00e863  2d7e28b4f79bcd9de2b3df0bc91310a3   
192      0189f417324cc706b727ae1944967c54  3eba0ec94cedb216bca77a07c6525d2d   
202      91aaaea6427f68f1338c6d3273304af2  a92320167ca0b39bea822c690e08c9d7   
...                                   ...                               ...   
7791652  a9ffeecb593fc6d7500ae1f565fda0f0  e1d97c03deb45d7cada039a08de92291   
7791670  353d9d983c3cd90068c59825622b5204  b3d3b112d865b3e99580b2ad49aca38d   
7791734  d2e569e2d1d8f7ad8d86d03ce8b534a0  5c366d4c5045ac3ff303191ddebaf95b   
7791744  fb2d2559a9e77372ffef7c9edebf87c5  c1b254a486172c86297e7a0af07981db   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [12]:
#筛选出train_agg_feat数据集中用户ID存在于测试集用户ID中的样本
train_agg_feat = train_agg_feat[train_agg_feat['user_id'].isin(testa_user['user_id'])]
print(train_agg_feat)

                                  user_id                          goods_id  \
191      c626a0405a9d85abdf05ccfd3d00e863  2d7e28b4f79bcd9de2b3df0bc91310a3   
705      060d0248975dfbe7dee433f32da5e266  ad182c11850f012948b84867362802be   
760      d01cfb26613e7ebfd53e8f50cfc2156a  5a6a5fb283c206e7d01effea87f68c8e   
800      9afec9408d028b77ab396251cbaac609  446430ddcdcf9a0cd80706a25ac7a45b   
819      96d46f4a620441197739c4d334364311  446430ddcdcf9a0cd80706a25ac7a45b   
...                                   ...                               ...   
7791220  9366b08f49ba3960eb6c58d41609cbd4  d805d53902739d835e6acac868f3ddbe   
7791271  884cfa576e9bd5e7c79307fdebd12ee4  73edfb067d6b0870214b7e51bfc630f5   
7791327  820b7afbd1f4e81a0fcbeee6d3123525  8805c368d93a74e0bdabe7832e8a9dee   
7791339  c37822c30576714979c146f01c7c5863  e82685998023f680ddb24cad402164f7   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [13]:
#将train_agg_feat数据集中用户ID存在于测试集用户ID中的样本筛选出来
train_agg_feat = train_agg_feat[train_agg_feat['goods_id'].isin(testa_goods['goods_id'])]
print(train_agg_feat)
train_agg_feat[['user_id', 'goods_id']].to_csv('u2i.csv', index=None)

                                  user_id                          goods_id  \
705      060d0248975dfbe7dee433f32da5e266  ad182c11850f012948b84867362802be   
760      d01cfb26613e7ebfd53e8f50cfc2156a  5a6a5fb283c206e7d01effea87f68c8e   
800      9afec9408d028b77ab396251cbaac609  446430ddcdcf9a0cd80706a25ac7a45b   
819      96d46f4a620441197739c4d334364311  446430ddcdcf9a0cd80706a25ac7a45b   
1164     de4997efdcdfa69819e1b0794815bb41  c46542bcd3fb900fa42d57d03cafb727   
...                                   ...                               ...   
7790636  672be194ab66116720214a875208541f  72a4e269893e6471c9d35f4ee0587251   
7791125  4bee2cd0eb5568b44929635e27ed98ea  0c8b7c2a8bca48ba289cc589aad748e0   
7791212  4019534b6bad8cdada154605bae3738f  009126a6f164ab270cb6c106b6a58028   
7791339  c37822c30576714979c146f01c7c5863  e82685998023f680ddb24cad402164f7   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [14]:
train_agg_feat = train_data.iloc[:].groupby(['user_id', 'goods_id']).agg({
    'is_clk': ['sum', 'max'],
    'is_like': ['sum', 'max'],
    'is_addcart': ['sum', 'max'],
    'is_order': ['sum', 'max'],
})
train_agg_feat = train_agg_feat.reset_index()
train_agg_feat.columns = [
    'user_id',
    'goods_id',
    'is_clk_sum',
    'is_clk_max',
    'is_like_sum',
    'is_like_max',
    'is_addcart_sum',
    'is_addcart_max',
    'is_order_sum',
    'is_order_max'
 ]

In [15]:
test_goods_id_agg = train_agg_feat.groupby('goods_id').agg({
    'is_clk_sum': 'sum',
    'is_order_max': 'sum'
})
test_goods_id_agg = test_goods_id_agg[test_goods_id_agg['is_clk_sum'] > 100]
test_valid_goods = test_goods_id_agg.index

In [16]:
train_feat_downsmaple = pd.concat([
    train_agg_feat[train_agg_feat['is_order_max'] !=0],
    train_agg_feat[train_agg_feat['is_order_max'] ==0].sample(int(0.03 * len(train_agg_feat)))
], axis=0)

In [17]:
1 - train_feat_downsmaple['is_order_max'].mean()

0.7795159078177217

In [18]:
train_feat_downsmaple['user_id'] = user_encode.transform(train_feat_downsmaple['user_id'])
train_feat_downsmaple['goods_id'] = goods_encode.transform(train_feat_downsmaple['goods_id'])

In [19]:
import paddle
import paddle.nn as nn
from paddle.io import Dataset

class SelfDefinedDataset(Dataset):
    def __init__(self, df, mode = 'train'):
        super(SelfDefinedDataset, self).__init__()
        self.df = df
        self.mode = mode

    def __getitem__(self, idx):
        if self.mode == 'predict':
            return (
                self.df['user_id'].iloc[idx],
                self.df['goods_id'].iloc[idx],
                self.df['is_clk_max'].iloc[idx], self.df['is_like_max'].iloc[idx],
                self.df['is_addcart_max'].iloc[idx],
            )
        else:
            return (
                self.df['user_id'].iloc[idx],
                self.df['goods_id'].iloc[idx],
                self.df['is_clk_max'].iloc[idx],
                (self.df['is_like_max'].iloc[idx]!= 0).astype('int64'),
                (self.df['is_addcart_max'].iloc[idx] != 0).astype('int64'),
                (self.df['is_order_max'].iloc[idx] != 0).astype('int64')
            )

    def __len__(self):
        return len(self.df)

In [20]:
train_feat_downsmaple['is_order_max'].max()

15

In [21]:
train_feat_downsmaple = train_feat_downsmaple.sample(frac=1.0)

traindataset = SelfDefinedDataset(train_feat_downsmaple.iloc[:-int(-0.2*len(train_feat_downsmaple))])
train_loader = paddle.io.DataLoader(traindataset, batch_size = 128, shuffle = True)

validdataset = SelfDefinedDataset(train_feat_downsmaple.iloc[-int(-0.2*len(train_feat_downsmaple)):])
valid_loader = paddle.io.DataLoader(validdataset, batch_size = 128, shuffle = True)

In [22]:
EMBEDDING_SIZE = 256

# 定义深度学习模型
class RecommenderNet(nn.Layer):
    def __init__(self, num_users, num_goods, embedding_size, numeric_size):
        super(RecommenderNet, self).__init__()
        self.num_users = num_users
        self.num_goods = num_goods
        self.embedding_size = embedding_size

        weight_attr_user = paddle.ParamAttr(
            regularizer = paddle.regularizer.L2Decay(1e-6),
            initializer = nn.initializer.KaimingNormal()
            )
        self.user_embedding = nn.Embedding(
            num_users,
            embedding_size,
            # weight_attr=weight_attr_user
        )

        weight_attr_movie = paddle.ParamAttr(
            regularizer = paddle.regularizer.L2Decay(1e-6),
            initializer = nn.initializer.KaimingNormal()
            )
        self.goods_embedding = nn.Embedding(
            num_goods,
            embedding_size,
            # weight_attr=weight_attr_movie
        )

        self.linear = nn.Linear(
            2 * embedding_size, 1
        )

    def forward(self, data):
        user, goods, feat = data[0], data[1], [data[idx] for idx in [2, 3,4]]
        feat = paddle.stack(feat, 1).astype(paddle.float32)

        user_vector = self.user_embedding(user)
        goods_vector = self.goods_embedding(goods)
        x = paddle.concat([user_vector, goods_vector], 1)
        return self.linear(x)

In [23]:
# 定义模型损失函数、优化器和评价指标
model = RecommenderNet(len(user_encode.classes_), len(goods_encode.classes_), 128, 3)
model

RecommenderNet(
  (user_embedding): Embedding(51602, 128, sparse=False)
  (goods_embedding): Embedding(3465659, 128, sparse=False)
  (linear): Linear(in_features=256, out_features=1, dtype=float32)
)

In [24]:
optimizer = paddle.optimizer.SGD(parameters=model.parameters(), learning_rate=0.01)
loss_fn = nn.BCEWithLogitsLoss()

In [25]:
# 模型训练
Train_Loss = []

for epoch in range(1):
    #将数据集的样本顺序进行洗牌
    train_feat_downsmaple = train_feat_downsmaple.sample(frac=1.0)
    #训练集为数据集的前90% 每个批次中的样本数量为128 在每个迭代中对数据进行随机洗牌
    traindataset = SelfDefinedDataset(train_feat_downsmaple.iloc[:-int(-0.1*len(train_feat_downsmaple))])
    train_loader = paddle.io.DataLoader(traindataset, batch_size = 128, shuffle = True)
    #验证集为数据集的后10% 每个批次中的样本数量为128 在每个迭代中对数据进行随机洗牌
    validdataset = SelfDefinedDataset(train_feat_downsmaple.iloc[-int(-0.1*len(train_feat_downsmaple)):])
    valid_loader = paddle.io.DataLoader(validdataset, batch_size = 128, shuffle = True)
    model.train()
    for i, data in enumerate(train_loader):
        pred = model(data)
        #计算损失值
        loss = loss_fn(data[-1].astype(paddle.float32), pred.flatten(), )
        #将损失值添加到训练损失列表中
        Train_Loss.append(loss.item())
        #根据损失值计算梯度
        loss.backward()
        #根据梯度更新模型参数
        optimizer.step()
        #清除梯度信息，为下一次迭代准备
        optimizer.clear_grad()

    # 模型验证
    Val_acc = []
    model.eval()
    with paddle.no_grad():
        for i, data in enumerate(valid_loader):
            pred = model(data)
            Val_acc.append(((pred.flatten() > 0).astype(paddle.int64) == data[-1]).sum().item())
    print(epoch, np.sum(Val_acc) / len(valid_loader.dataset))

0 0.5040652151516192


In [ ]:
test_valid_goods = test_valid_goods[test_valid_goods.isin(test_valid_goods)]
test_feat_agg = []
for user in testa_user['user_id'].values:
    for goods in test_valid_goods:
        test_feat_agg.append([user, goods])
test_feat_agg = pd.DataFrame(test_feat_agg)
test_feat_agg.columns = ['user_id', 'goods_id']
test_feat_agg = pd.merge(test_feat_agg, train_agg_feat, on=['user_id', 'goods_id'])
test_feat_agg['user_id'] = user_encode.transform(test_feat_agg['user_id'])
test_feat_agg['goods_id'] = goods_encode.transform(test_feat_agg['goods_id'])
testdataset = SelfDefinedDataset(test_feat_agg)
test_loader = paddle.io.DataLoader(testdataset, batch_size = 128, shuffle = False)
model.eval()

test_pred = []
with paddle.no_grad():
    for i, data in enumerate(test_loader):
        pred = model(data)
        test_pred += list((paddle.nn.functional.sigmoid(pred) > 0.5).numpy().reshape(-1))
test_feat_agg['pred'] = test_pred